In [ ]:
!pip install ir-evaluation-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import json
import operator
from ir_evaluation.effectiveness import effectiveness
import random
import time
import ast
from google.colab import drive
import matplotlib.pyplot as plt
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
[i for i in nltk.word_tokenize("what are the structural and aeroelastic problems associated with flight of high speed aircraft .") if i not in stopwords.words('english')]

['structural',
 'aeroelastic',
 'problems',
 'associated',
 'flight',
 'high',
 'speed',
 'aircraft',
 '.']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def serialize_sets(obj):
    if isinstance(obj, set):
        return list(obj)

    return obj

## **IMPORT CRANFIELD 1400 DATASET (giải nén 1400 file cranfield được lưu trong /cran/cran.all.1400)**

In [ ]:
def text_process(content):
    stop_list = stopwords.words('english')
    stem = PorterStemmer()

    content = content.lstrip()
    content = re.sub('[^A-Za-z]+', ' ', content)
    content = content.lower()
    tokens = nltk.word_tokenize(content)

    stem_tokens = [stem.stem(word) for word in tokens if word not in stop_list]
    clean_stem_tokens = ' '.join(map(str,  stem_tokens))

    shortword = re.compile(r'\W*\b\w{1,2}\b')
    clean_stem_tokens = shortword.sub('', clean_stem_tokens)

    return clean_stem_tokens

In [ ]:
content = "what problems of heat conduction in composite slabs have been solved so far ."
text_process(content)

'problem heat conduct composit slab solv far'

Create a folder to stores files of Cranfield dataset (cran/)
For each dataset, create folders to store processed data:
- docs_files: a folder to store each raw doc in separated file
- processed_docs: a folder to store processed doc in separated file
- query_files: a folder to store each raw query in separated file
- vsm_result: a folder to store ranked list of VSM model of each doc in separated file
- prob_result: a folder to store ranked list of VSM model of each doc in separated file

In [ ]:
# store each doc in a seperated file
with open('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/cran.all.1400', 'r') as f:
    content = f.readlines()
    for line in content:
      if line.startswith('.I'):
          f_ = line.split('.I')[1]
          name = '/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/doc_files/' + f_[:-1].strip() + '.txt'
          f1 = open(name, 'w')
          f1.write(line)
          f1.close()
      else:
          with open(name, 'a') as f1:
              f1.write(line)
          f1.close()
    f.close()

In [ ]:
# save processed doc
dir = os.listdir('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/doc_files/')
for d in dir:
    with open('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/doc_files/' + d, 'r') as f:
        content = f.readlines()
        flag = False
        content_ = ''
        for line in content:
            if line.startswith('.W'):
                flag = True
            if flag == True:
                if line.startswith('.W'):
                    continue
                else:
                    content_ = content_ + line
        new_content = text_process(content_)
        with open('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/processed_doc/' + d, 'w') as f1:
            f1.write(new_content)
            f1.close()
    f.close()

# **INDEXING (LẬP CHỈ MỤC)**

In [ ]:
content = "Today is Saturday and I am feeling so relaxed"
tokens = nltk.word_tokenize(content)
print(tokens)

['Today', 'is', 'Saturday', 'and', 'I', 'am', 'feeling', 'so', 'relaxed']


In [ ]:
dictionary = {"1": 1, "2": 2}
print("1" in dictionary, "3" in dictionary)

True False


**Dưới đây là hàm lập chỉ mục dựa theo mô hình không gian vector. Các biến quan trọng gồm có:**

*   **inverted_index: là một dictionary với mỗi key là một term, còn value là một dictionary khác mà mỗi cặp {key: value} có dạng {tài_liệu: trọng_số_tf*idf của term trong tài_liệu}**

In [ ]:
def create_vector_index(path):
    inverted_index = {}    # for each term, its w = tf*idf in each document will be saved into this dictionary
    dir = os.listdir(path) # list of links to documents
    no_docs = len(dir)    # number of documents
    for d in dir:
        id = d.split('.')[0] # document's name
        f = open(path + d, 'r')
        content = f.read()  # reading content from a document
        f.close()
        tokens = nltk.word_tokenize(content)
        for token in tokens:
            if token not in inverted_index:
                inverted_index[token] = {}
            if id not in inverted_index[token]:
                inverted_index[token][id] = [0, 0]
                inverted_index[token][id][0] = 1
            else:
                inverted_index[token][id][0] += 1
    weights = {}
    idf_dict = {}
    for term in inverted_index:       # term ~ token
        idf = np.log(no_docs/len(inverted_index[term]))
        idf_dict[term] = idf
        for id in inverted_index[term]:
            if id not in weights:
                weights[id] = []
            inverted_index[term][id][1] = (1 + np.log(inverted_index[term][id][0]))*idf # calculate w = tf*idf
            weights[id].append(inverted_index[term][id][1])

    # WEIGHT NORMALIZATION
    weights_norm = {}
    for id in weights:
        sqr = np.square(weights[id])
        sum_sqr = np.sum(sqr)
        norm = np.sqrt(sum_sqr)
        weights_norm[id] = norm
    for term in inverted_index:
        for id in inverted_index[term]:
            inverted_index[term][id][1] = inverted_index[term][id][1]/weights_norm[id]
    return idf_dict, inverted_index

In [ ]:
index_file = "/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/inverted_index_vect.txt"
f = open(index_file, "r")
contents = f.read()
index_dict = ast.literal_eval(contents)

In [ ]:
for i in index_dict["inviscid"].items():
    if i[1] == [0, 0]:
      print(i[0])

## ĐẾM THỜI GIAN LẬP CHỈ MỤC SỬ DỤNG VECTOR SPACE MODEL

**Vector Space Model**

In [ ]:
start_time = time.time()
idf_dict, index = create_vector_index('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/processed_doc/')
f = open('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/inverted_index_vect.txt', 'w')
f.write(json.dumps(index))
f.close()
f = open('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/idf_dict.txt', 'w')
f.write(json.dumps(idf_dict))
f.close()
print(time.time() - start_time)

28.06743359565735


# **COSINE SIMILARITY**

In [ ]:
def process_query(qry):
    return text_process(qry)

In [ ]:
qry = '''is it possible to relate the available pressure distributions for an
ogive forebody at zero angle of attack to the lower surface pressures of
an equivalent ogive forebody at angle of attack .'''

In [ ]:
increase_font()
process_query(qry)

<IPython.core.display.Javascript object>

'possibl relat avail pressur distribut ogiv forebodi zero angl attack lower surfac pressur equival ogiv forebodi angl attack'

In [ ]:
def read_index(index_path):
    f = open(index_path, 'r')
    data = f.read()
    index = json.loads(data)
    f.close()

    return index

In [ ]:
def form_weight_dict(qry, inverted_index, idf_dict):
  weight_dict = {}
  tokens = nltk.word_tokenize(qry)
  for token in tokens:
    if token in inverted_index:
      if token not in weight_dict:
        weight_dict[token] = 1
      else:
        weight_dict[token] += 1
  for token in weight_dict:
    weight_dict[token] = (1 + np.log(weight_dict[token])) * idf_dict[token]
  for term in inverted_index:
    if term not in weight_dict:
      weight_dict[term] = 0

  # WEIGHT NORMALIZATION
  norm = np.sqrt(np.sum(np.square(list(weight_dict.values()))))
  for term in weight_dict:
    if norm != 0:
      weight_dict[term] = weight_dict[term] / norm
  return weight_dict

In [ ]:
def cos(weight_dict, inverted_index):
  inv_ind_normed = {}
  for term in inverted_index:
    sum = 0
    for doc_id in inverted_index[term]:
      sum += np.square(inverted_index[term][doc_id][1])
    inv_ind_normed[term] = np.sqrt(sum)

  weight_dict_normed = {}
  norm = np.sqrt(np.sum(np.square(list(weight_dict.values()))))
  for term in weight_dict:
    if norm != 0:
      weight_dict_normed[term] = weight_dict[term] / norm

  sums = {}
  for doc_id in range(1, 1401):
    sum = 0
    for term in weight_dict:
      if str(doc_id) not in inverted_index[term]:
        continue
      sum += weight_dict_normed[term] * (inverted_index[term][str(doc_id)][1]/inv_ind_normed[term])
    sums[str(doc_id)] = sum
  return sums

# **RETRIEVAL**

**Hàm truy vấn theo mô hình Vector Space**

In [ ]:
#save each query in seperate file
f = open('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/cran.qry', 'r')
lines = f.readlines()
f.close()
flag = False
count = 1
for line in lines:
    if line.startswith('.I'):
        if flag == True:
            f = open('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/query_files/' + qry_id + '.txt', 'w')
            f.write(qry)
            f.close()
        qry_id = str(count)
        qry = ''
        count += 1
        flag = False
    elif line.startswith('.W'):
        flag = True
    else: qry += line
f = open('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/query_files/' + qry_id + '.txt', 'w')
f.write(qry)
f.close()

In [ ]:
start_time = time.time()
index = read_index('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/inverted_index_vect.txt')
query_list = os.listdir('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/query_files/')
for name in query_list:
    result = {}
    qry_id = name.split('.')[0]
    f = open('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/query_files/' + name, 'r')
    qry = f.read()
    f.close()
    qry = process_query(qry)
    weight_dict = form_weight_dict(qry, index, idf_dict)
    f = open('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/qry_weight_vector/' + name, 'w')
    f.write(json.dumps(weight_dict))
    f.close()
    result = cos(weight_dict, index)
    f = open('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/vector_result/' + name, 'w')
    f.write(json.dumps(result))
    f.close()
print(time.time() - start_time)

553.4899916648865


In [ ]:
new_query = " clear survey duration"
new_query = process_query(new_query)
new_weight_dict = form_weight_dict(new_query, index, idf_dict)
result = cos(new_weight_dict, index)
f = open('/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/new_qry_result.txt', 'w')
f.write(json.dumps(result))
f.close()

# **DOCUMENT RANKING**

In [ ]:
def increase_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '20px'
      break
    }
  }
  '''))

In [ ]:
def rank(query):
  qry_result_file = "/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/vector_result/" + str(query) + ".txt"
  f = open(qry_result_file, "r")
  contents = f.read()
  cos_sim = ast.literal_eval(contents)
  f.close()
  sorted_docs = sorted(cos_sim.items(), key=lambda x: x[1], reverse=True)
  return sorted_docs

In [ ]:
ranking = rank(7)

In [ ]:
ranking_dict = {"Tài liệu": [], "Độ tương đồng": []}
for i in ranking:
  ranking_dict["Tài liệu"].append(i[0])
  ranking_dict["Độ tương đồng"].append(i[1])
ranking_df = pd.DataFrame(ranking_dict)

In [ ]:
increase_font()
print("Xếp hạng các tài liệu khi thực hiện truy vấn với query thứ 7")
ranking_df.head(7)

<IPython.core.display.Javascript object>

Xếp hạng các tài liệu khi thực hiện truy vấn với query thứ 7


,Tài liệu,Độ tương đồng
0,492,0.821812
1,973,0.376403
2,434,0.356496
3,233,0.283985
4,57,0.258576
5,124,0.233845
6,56,0.226599


# **PRECISION, RECALL AND MAP**

In [ ]:
res_1 = "/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/RES/1.txt"
f = open(res_1, "r")
contents = f.readlines()
print(len(contents))

29


In [ ]:
def prec_rec(qry_id):
  ranking = rank(qry_id)
  rel_file = "/content/drive/MyDrive/Colab Notebooks/Information Retrieval/Final-project/cran/RES/" + str(qry_id) + ".txt"
  f = open(rel_file, "r")
  lines = f.readlines()
  rel_no = len(lines)
  A = [j.split()[1] for j in lines]
  f.close()
  B = [i[0] for i in ranking if i[1] > 0]
  returned_no = len(B)
  rel_returned_no = len(set(A) & set(B))
  rec = rel_returned_no / rel_no
  prec = rel_returned_no / returned_no
  return prec, rec

In [ ]:
prec_rec_list = [prec_rec(qry_id) for qry_id in range(1, 226)]
avg_prec = sum(i[0] for i in prec_rec_list) / 225
avg_rec = sum(i[1] for i in prec_rec_list) / 225

In [ ]:
print("Average precision for 225 queries: ", avg_prec)
print("Average recall for 225 queries: ", avg_rec)

Average precision for 225 queries:  0.009636555778657558
Average recall for 225 queries:  0.9608491963124148
